In [4]:
import zipfile
import os
!pip install tensorflow
import tensorflow as tf

# Define the base directory for extraction
dataset_base_dir = 'dataset'

# Provide FULL paths to your zip files here
zip_files = [
    'Coccinella_novemnotata.zip',
    'Harmonia_axyridis.zip'
]

# Create the base dataset directory if it doesn't exist
os.makedirs(dataset_base_dir, exist_ok=True)
print(f"Created base directory: {dataset_base_dir}")

# Process each zip file
for zip_file_path in zip_files:
    # Get the filename only (e.g., 'Harmonia_axyridis.zip')
    zip_file_name = os.path.basename(zip_file_path)

    # Determine the extraction directory (remove .zip)
    extraction_dir_name = os.path.splitext(zip_file_name)[0]

    # Full output path
    extraction_path = os.path.join(dataset_base_dir, extraction_dir_name)
    os.makedirs(extraction_path, exist_ok=True)
    
    print(f"Created extraction directory: {extraction_path}")

    # Extract files
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    print(f"Extracted '{zip_file_name}' to '{extraction_path}'")

print("All zip files extracted successfully.")

Defaulting to user installation because normal site-packages is not writeable


2025-11-19 12:40:17.910320: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-19 12:40:18.095209: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 12:40:20.895422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Created base directory: dataset
Created extraction directory: dataset/Coccinella_novemnotata
Extracted 'Coccinella_novemnotata.zip' to 'dataset/Coccinella_novemnotata'
Created extraction directory: dataset/Harmonia_axyridis
Extracted 'Harmonia_axyridis.zip' to 'dataset/Harmonia_axyridis'
All zip files extracted successfully.


In [5]:
import tensorflow as tf

# 1. Define image dimensions and batch size
IMG_HEIGHT = 180
IMG_WIDTH = 180
BATCH_SIZE = 32

# 2. Load the entire dataset
# The dataset directory should now contain 'Coccinella_novemnotata' and 'Harmonia_axyridis' subdirectories
# which contain the actual images.
# We need to specify the path to the subdirectories containing the images.
# Since the previous extraction created a nested structure like dataset/Coccinella_novemnotata/Coccinella_novemnotata
# we need to point image_dataset_from_directory to the parent folder of these two, which is 'dataset'.
image_dataset = tf.keras.utils.image_dataset_from_directory(
    'dataset',
    labels='inferred',
    label_mode='int',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=True # Shuffle for randomness
)

# Get class names
class_names = image_dataset.class_names
print(f"Found {len(class_names)} classes: {class_names}")

# 3. Determine the number of training images (80% of total batches)
# It's generally better to split the dataset BEFORE batching to ensure a more precise split,
# but image_dataset_from_directory returns a batched dataset. We will split at the batch level.
# total_batches = tf.data.experimental.cardinality(image_dataset).numpy()
# Let's get the number of elements first. This requires iterating, which is not ideal.
# A more robust way is to determine the split percentage on the total number of items, not batches
# However, since image_dataset_from_directory returns batched data, we will split based on batches.

# The total number of batches is unknown directly without iterating or using `cardinality()`
# Let's use `cardinality()` which is preferred
dataset_size = tf.data.experimental.cardinality(image_dataset)
if dataset_size == tf.data.experimental.UNKNOWN_CARDINALITY:
    print("Warning: Dataset cardinality is unknown. Estimating split based on a sample of batches.")
    # Fallback for unknown cardinality: take a few batches to estimate
    # This is not ideal for precise splits but works if direct cardinality is not available
    num_batches_to_estimate = 10
    estimated_total_batches = 0
    for _ in image_dataset.take(num_batches_to_estimate):
        estimated_total_batches += 1
    if estimated_total_batches < num_batches_to_estimate:
        # Less than num_batches_to_estimate batches, so this is the true total
        total_batches = estimated_total_batches
    else:
        # Cannot reliably determine total batches without full iteration, assume a large enough number for now
        # For this exercise, we will assume cardinality works or a rough split is acceptable.
        # To get an accurate count, you'd need to iterate or load all into memory.
        # Given the subtask, `cardinality()` should work for actual batch count.
        # Let's re-run image_dataset_from_directory without batching to get total elements if necessary,
        # or iterate once to count if cardinality doesn't work well.
        # For now, let's assume cardinality works for the purpose of the notebook.
        # If not, a more robust solution would be to use tf.data.Dataset.from_generator after counting files.
        # However, sticking to the instruction to use image_dataset_from_directory.
        pass # Rely on next block if cardinality is not unknown

if dataset_size != tf.data.experimental.UNKNOWN_CARDINALITY:
    total_batches = dataset_size.numpy()
else:
    # If cardinality is still unknown after initial attempts, iterate to find total batches
    print("Iterating through dataset to determine total number of batches...")
    total_batches = 0
    for _ in image_dataset:
        total_batches += 1
    print(f"Determined total number of batches: {total_batches}")

train_batches = int(total_batches * 0.8)

# 4. Split the image_dataset into training and validation sets
train_ds = image_dataset.take(train_batches)
remaining_ds = image_dataset.skip(train_batches)

# The remaining_ds is 20% of the original data, which will serve as our test set.
test_ds = remaining_ds

print(f"Total batches: {total_batches}")
print(f"Training batches: {tf.data.experimental.cardinality(train_ds).numpy()}")
print(f"Test batches: {tf.data.experimental.cardinality(test_ds).numpy()}")

# 5. Apply data caching and prefetching
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("Dataset loading and splitting complete. Data has been cached and prefetched.")

Found 240 files belonging to 2 classes.
Found 2 classes: ['Coccinella_novemnotata', 'Harmonia_axyridis']
Total batches: 8
Training batches: 6
Test batches: 2
Dataset loading and splitting complete. Data has been cached and prefetched.


2025-11-19 12:40:26.297994: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Rescaling

# Define the CNN model architecture
model = Sequential([
    Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)), # Normalize pixel values
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='sigmoid') # Binary classification for 2 classes
])

# Print the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 178, 178, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 89, 89, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 87, 87, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 43, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 41, 41, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,276,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,300,770 (12.59 MB)

 Trainable params: 3,300,770 (12.59 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.optimizers import Adam

# 1. Compile the model
# For binary classification with two classes, where labels are integers (0 or 1),
# and the output layer has 2 units with 'sigmoid' activation, BinaryCrossentropy is suitable
# if the labels were one-hot encoded, but since label_mode='int' was used, SparseCategoricalCrossentropy is appropriate.
# However, the current model architecture (last layer Dense(2) with sigmoid) implies two independent binary predictions.
# If it's a true binary classification (class 0 or class 1), the final Dense layer should have 1 unit with 'sigmoid'.
# Given the current setup with 2 classes and labels='int', SparseCategoricalCrossentropy is the correct loss function.
# Let's adjust the final dense layer to be 1 unit and use binary crossentropy, which is more common for 2 classes.

# Re-define the model with 1 output unit for binary classification
model_redefined = Sequential([
    Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)), # Normalize pixel values
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid') # 1 output unit for binary classification
])

model_redefined.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully with Adam optimizer and binary_crossentropy loss.")
model_redefined.summary()

# 2. Train the model
EPOCHS = 10
history = model_redefined.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS
)

print(f"Model trained for {EPOCHS} epochs.")

Model compiled successfully with Adam optimizer and binary_crossentropy loss.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 178, 178, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 89, 89, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 87, 87, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 43, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 41, 41, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     3,276,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,300,641 (12.59 MB)

 Trainable params: 3,300,641 (12.59 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 442ms/step - accuracy: 0.5052 - loss: 1.1601 - val_accuracy: 0.5625 - val_loss: 0.7033
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - accuracy: 0.5365 - loss: 0.7087 - val_accuracy: 0.4583 - val_loss: 0.6920
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - accuracy: 0.5156 - loss: 0.6908 - val_accuracy: 0.6250 - val_loss: 0.6824
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 222ms/step - accuracy: 0.6146 - loss: 0.6818 - val_accuracy: 0.5833 - val_loss: 0.6600
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.6458 - loss: 0.6584 - val_accuracy: 0.6458 - val_loss: 0.6141
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - accuracy: 0.6354 - loss: 0.6313 - val_accuracy: 0.6250 - val_loss: 0.5855
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - accuracy: 0.6927 - loss: 0.5765 - val_accuracy: 0.6458 - val_loss: 0.5560
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 222ms/step - accuracy: 0.7604 - loss: 0.4819 - val_accuracy: 0.7708 - val_loss:

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

# 1. Compile the model
# For binary classification with two classes, where labels are integers (0 or 1),
# and the output layer has 2 units with 'sigmoid' activation, BinaryCrossentropy is suitable
# if the labels were one-hot encoded, but since label_mode='int' was used, SparseCategoricalCrossentropy is appropriate.
# However, the current model architecture (last layer Dense(2) with sigmoid) implies two independent binary predictions.
# If it's a true binary classification (class 0 or class 1), the final Dense layer should have 1 unit with 'sigmoid'.
# Given the current setup with 2 classes and labels='int', SparseCategoricalCrossentropy is the correct loss function.
# Let's adjust the final dense layer to be 1 unit and use binary crossentropy, which is more common for 2 classes.

# Re-define the model with 1 output unit for binary classification
model_redefined = Sequential([
    Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)), # Explicitly define Input layer
    Rescaling(1./255), # Normalize pixel values, no input_shape needed here
    Conv2D(16, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid') # 1 output unit for binary classification
])

model_redefined.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully with Adam optimizer and binary_crossentropy loss.")
model_redefined.summary()

# 2. Train the model
EPOCHS = 10
history = model_redefined.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS
)

print(f"Model trained for {EPOCHS} epochs.")

Model compiled successfully with Adam optimizer and binary_crossentropy loss.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_3 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 178, 178, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 89, 89, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 87, 87, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 43, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 41, 41, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     3,276,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,300,641 (12.59 MB)

 Trainable params: 3,300,641 (12.59 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 248ms/step - accuracy: 0.5052 - loss: 1.5765 - val_accuracy: 0.5625 - val_loss: 0.6891
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.5052 - loss: 0.7060 - val_accuracy: 0.4375 - val_loss: 0.6960
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 218ms/step - accuracy: 0.5417 - loss: 0.6922 - val_accuracy: 0.6250 - val_loss: 0.6820
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 218ms/step - accuracy: 0.6146 - loss: 0.6832 - val_accuracy: 0.5833 - val_loss: 0.6667
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 228ms/step - accuracy: 0.6042 - loss: 0.6645 - val_accuracy: 0.6458 - val_loss: 0.6365
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 216ms/step - accuracy: 0.6615 - loss: 0.6318 - val_accuracy: 0.6250 - val_loss: 0.6034
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 221ms/step - accuracy: 0.6771 - loss: 0.5969 - val_accuracy: 0.6458 - val_loss: 0.5899
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 228ms/step - accuracy: 0.7708 - loss: 0.5351 - val_accuracy: 0.7083 - val_loss:

In [10]:
print("\nEvaluating the model on the test dataset...")
loss, accuracy = model_redefined.evaluate(test_ds)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Evaluating the model on the test dataset...
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7917 - loss: 0.4895
Test Loss: 0.4895
Test Accuracy: 0.7917


In [11]:
import numpy as np

# Extract validation accuracy from the history object
validation_accuracy = history.history['val_accuracy']

# Print all validation accuracies per epoch
print("Validation accuracy per epoch:")
for i, acc in enumerate(validation_accuracy):
    print(f"Epoch {i+1}: {acc:.4f}")

# Calculate and print the average validation accuracy
average_val_accuracy = np.mean(validation_accuracy)
print(f"\nAverage Validation Accuracy: {average_val_accuracy:.4f}")

Validation accuracy per epoch:
Epoch 1: 0.5625
Epoch 2: 0.4375
Epoch 3: 0.6250
Epoch 4: 0.5833
Epoch 5: 0.6458
Epoch 6: 0.6250
Epoch 7: 0.6458
Epoch 8: 0.7083
Epoch 9: 0.7292
Epoch 10: 0.7917

Average Validation Accuracy: 0.6354


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Rescaling, Input

# 1. Define ranges for hyperparameters
epochs_range = [5, 10]
batch_sizes_range = [16, 32]

# 2. Initialize an empty list to store results
results = []

print("Starting hyperparameter tuning...")

# Outer loop for batch sizes
for current_batch_size in batch_sizes_range:
    print(f"\n--- Testing with Batch Size: {current_batch_size} ---")

    # 4. Re-load the dataset with the current batch size
    image_dataset_tuned = tf.keras.utils.image_dataset_from_directory(
        'dataset',
        labels='inferred',
        label_mode='int',
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        interpolation='nearest',
        batch_size=current_batch_size,
        shuffle=True
    )

    # 5. Re-split the dataset into training and testing sets
    total_batches_tuned = tf.data.experimental.cardinality(image_dataset_tuned).numpy()
    train_batches_tuned = int(total_batches_tuned * 0.8)

    train_ds_tuned = image_dataset_tuned.take(train_batches_tuned)
    test_ds_tuned = image_dataset_tuned.skip(train_batches_tuned)

    print(f"Total batches for batch_size {current_batch_size}: {total_batches_tuned}")
    print(f"Training batches: {tf.data.experimental.cardinality(train_ds_tuned).numpy()}")
    print(f"Test batches: {tf.data.experimental.cardinality(test_ds_tuned).numpy()}")

    # 6. Apply caching and prefetching
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds_tuned = train_ds_tuned.cache().prefetch(buffer_size=AUTOTUNE)
    test_ds_tuned = test_ds_tuned.cache().prefetch(buffer_size=AUTOTUNE)

    # Inner loop for epochs
    for current_epochs in epochs_range:
        print(f"\n-- Training for {current_epochs} Epochs --")

        # 8. Re-define the model for a fresh start
        model_tuned = Sequential([
            Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
            Rescaling(1./255),
            Conv2D(16, 3, activation='relu'),
            MaxPooling2D(),
            Conv2D(32, 3, activation='relu'),
            MaxPooling2D(),
            Conv2D(64, 3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(1, activation='sigmoid') # Binary classification
        ])

        # 9. Compile the model
        model_tuned.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        # 10. Train the model
        history_tuned = model_tuned.fit(
            train_ds_tuned,
            validation_data=test_ds_tuned,
            epochs=current_epochs,
            verbose=0 # Suppress verbose output during tuning for cleaner logs
        )
        print(f"Model trained for {current_epochs} epochs.")

        # 11. Evaluate the trained model
        loss_tuned, accuracy_tuned = model_tuned.evaluate(test_ds_tuned, verbose=0)
        print(f"Evaluation - Test Loss: {loss_tuned:.4f}, Test Accuracy: {accuracy_tuned:.4f}")

        # 12. Store the results
        results.append({
            'batch_size': current_batch_size,
            'epochs': current_epochs,
            'train_accuracy': history_tuned.history['accuracy'][-1] if 'accuracy' in history_tuned.history else 'N/A',
            'val_accuracy': history_tuned.history['val_accuracy'][-1] if 'val_accuracy' in history_tuned.history else 'N/A',
            'test_loss': loss_tuned,
            'test_accuracy': accuracy_tuned
        })

print("Hyperparameter tuning complete.")
print("Summary of results:")
for r in results:
    print(r)

Starting hyperparameter tuning...

--- Testing with Batch Size: 16 ---
Found 240 files belonging to 2 classes.
Total batches for batch_size 16: 15
Training batches: 12
Test batches: 3

-- Training for 5 Epochs --
Model trained for 5 epochs.
Evaluation - Test Loss: 0.6511, Test Accuracy: 0.7083

-- Training for 10 Epochs --
Model trained for 10 epochs.
Evaluation - Test Loss: 0.3290, Test Accuracy: 0.9167

--- Testing with Batch Size: 32 ---
Found 240 files belonging to 2 classes.
Total batches for batch_size 32: 8
Training batches: 6
Test batches: 2

-- Training for 5 Epochs --
Model trained for 5 epochs.
Evaluation - Test Loss: 0.5017, Test Accuracy: 0.8125

-- Training for 10 Epochs --
Model trained for 10 epochs.
Evaluation - Test Loss: 0.3162, Test Accuracy: 0.9167
Hyperparameter tuning complete.
Summary of results:
{'batch_size': 16, 'epochs': 5, 'train_accuracy': 0.7447916865348816, 'val_accuracy': 0.7083333134651184, 'test_loss': 0.6511179804801941, 'test_accuracy': 0.7083333134